In [814]:
# Libraries

# EDA 
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# -Pre-Processing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# -Model Selection
from sklearn.model_selection import train_test_split


# -Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Model validation
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve 

#Hyper-parameter Hyper-parameter
from sklearn.model_selection import GridSearchCV

# -Metrics
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report

%matplotlib inline

In [815]:

#Import of very relevant data 

course_ratings = pd.read_csv('365_database/365_course_ratings.csv')

student_info = pd.read_csv('365_database/365_student_info.csv')

student_learning = pd.read_csv('365_database/365_student_learning.csv')

student_purchases = pd.read_csv('365_database/365_student_purchases.csv')

# Less relevant data

#course_info = pd.read_csv('365_database/365_course_info.csv')

#exam_info = pd.read_csv('365_database/365_exam_info.csv')

#quiz_info = pd.read_csv('365_database/365_quiz_info.csv')

#student_engagement = pd.read_csv('365_database/365_student_engagement.csv')

#student_exams = pd.read_csv('365_database/365_student_exams.csv')

#student_hub_questions = pd.read_csv('365_database/365_student_hub_questions.csv')

#student_quizzes =pd.read_csv('365_database/365_student_quizzes.csv')


# Work process :

<ol>
    <li>
        Dataset preparation and preprocessing
        <ul>
            <li>Data collection</li>
            <li>Data visualization</li>
            <li>Data selection</li>
            <li>Dataset splitting</li>
            <li>Data Cleaning</li>
            <li>Data formatting</li>
            <li>Data Scaling</li>
        </ul>
    </li>
    <li>Modeling and Validation</li>
    <li>Evaluating the model using  test set</li>
</ol>

# Dataset preparation and preprocessing

## Data Collection

In [816]:
# A glance at the various data

In [817]:
course_ratings.head()

,course_id,student_id,course_rating,date_rated
0,14,258956,5,2022-07-06
1,7,259019,5,2022-08-30
2,23,259019,4,2022-08-30
3,14,259283,5,2022-07-08
4,30,259283,5,2022-07-02


In [818]:
student_info.head()

,student_id,student_country,date_registered
0,258798,IN,2022-01-01
1,258799,CO,2022-01-01
2,258800,CA,2022-01-01
3,258801,IN,2022-01-01
4,258802,US,2022-01-01


In [819]:
student_learning.head()

,student_id,course_id,minutes_watched,date_watched
0,258798,23,0.3,2022-01-01
1,258800,2,12.9,2022-04-01
2,258800,7,46.8,2022-01-03
3,258800,7,4.3,2022-01-04
4,258800,7,31.7,2022-01-07


In [820]:
student_purchases.head()

,purchase_id,student_id,purchase_type,date_purchased
0,15781,258800,Annual,2022-01-01
1,15786,258803,Annual,2022-01-01
2,15808,258862,Annual,2022-01-01
3,15809,258865,Annual,2022-01-01
4,15811,258878,Annual,2022-01-01


In [821]:
# Merge differents data to build a data set 

In [822]:
df = pd.merge(student_learning, student_purchases, how='left', on='student_id').merge(student_info, how='left', on='student_id').merge(course_ratings, how='left', on=['student_id', 'course_id'])

In [823]:
df.head()

,student_id,course_id,minutes_watched,date_watched,purchase_id,purchase_type,date_purchased,student_country,date_registered,course_rating,date_rated
0,258798,23,0.3,2022-01-01,NaN,NaN,NaN,IN,2022-01-01,NaN,NaN
1,258800,2,12.9,2022-04-01,15781.0,Annual,2022-01-01,CA,2022-01-01,NaN,NaN
2,258800,7,46.8,2022-01-03,15781.0,Annual,2022-01-01,CA,2022-01-01,NaN,NaN
3,258800,7,4.3,2022-01-04,15781.0,Annual,2022-01-01,CA,2022-01-01,NaN,NaN
4,258800,7,31.7,2022-01-07,15781.0,Annual,2022-01-01,CA,2022-01-01,NaN,NaN


In [824]:
df['purchase'] = df['purchase_type']

df.drop('purchase_type', axis=1, inplace=True)

dic = {'Annual':1, 'Monthly':1, 'Quarterly': 1, np.nan: 0}

df['purchase'] = df['purchase'].map(dic)

df['purchase'] = df['purchase'].astype('int')

In [825]:
df.head(10)

,student_id,course_id,minutes_watched,date_watched,purchase_id,date_purchased,student_country,date_registered,course_rating,date_rated,purchase
0,258798,23,0.3,2022-01-01,NaN,NaN,IN,2022-01-01,NaN,NaN,0
1,258800,2,12.9,2022-04-01,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
2,258800,7,46.8,2022-01-03,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
3,258800,7,4.3,2022-01-04,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
4,258800,7,31.7,2022-01-07,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
5,258800,7,61.6,2022-01-05,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
6,258800,7,45.1,2022-01-06,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
7,258800,11,0.5,2022-02-21,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
8,258800,11,1.1,2022-02-22,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1
9,258800,14,4.0,2022-01-28,15781.0,2022-01-01,CA,2022-01-01,NaN,NaN,1


In [826]:
df.shape

(87633, 11)

In [827]:
# the dataset has 87633 sample, not bad 😋 and 12 columns

In [828]:
df.isnull().sum()

student_id             0
course_id              0
minutes_watched        0
date_watched           0
purchase_id        27204
date_purchased     27204
student_country       44
date_registered        0
course_rating      71054
date_rated         71054
purchase               0
dtype: int64

In [829]:
# A lot of missing values, this is normal due to the merge 😐

In [830]:
df.dtypes

student_id           int64
course_id            int64
minutes_watched    float64
date_watched        object
purchase_id        float64
date_purchased      object
student_country     object
date_registered     object
course_rating      float64
date_rated          object
purchase             int32
dtype: object

##### Remarks

In [831]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
student_id,87633.0,272453.644597,9934.821750,258798.0,263690.0,270486.0,280019.0,295513.0
course_id,87633.0,19.587941,12.274264,2.0,7.0,16.0,24.0,67.0
minutes_watched,87633.0,31.387779,41.174484,0.0,5.4,18.9,42.2,1710.6
purchase_id,60429.0,19469.821112,2054.504091,15781.0,17677.0,19504.0,21243.0,23152.0
course_rating,16579.0,4.795766,0.520219,1.0,5.0,5.0,5.0,5.0
purchase,87633.0,0.689569,0.462673,0.0,0.0,1.0,1.0,1.0


##### Remarks

- Minutes watched 
    *50% of students watched videos of more than 10 min
- Rating Course
    *Most of the students give, This is normal, since the quality of teaching is excellent

## Data Visualisation

In [832]:
df['student_country'].value_counts()

US    22180
IN     8986
GB     4979
CA     4544
NG     3717
      ...  
PG        1
XK        1
KM        1
DM        1
CG        1
Name: student_country, Length: 173, dtype: int64

## Data selection

In [833]:
data = df.copy()
data = df.drop(['purchase_id', 'student_id', 'date_purchased'], axis=1)

In [834]:
data.head()

,course_id,minutes_watched,date_watched,student_country,date_registered,course_rating,date_rated,purchase
0,23,0.3,2022-01-01,IN,2022-01-01,NaN,NaN,0
1,2,12.9,2022-04-01,CA,2022-01-01,NaN,NaN,1
2,7,46.8,2022-01-03,CA,2022-01-01,NaN,NaN,1
3,7,4.3,2022-01-04,CA,2022-01-01,NaN,NaN,1
4,7,31.7,2022-01-07,CA,2022-01-01,NaN,NaN,1


In [835]:
data.isna().sum()

course_id              0
minutes_watched        0
date_watched           0
student_country       44
date_registered        0
course_rating      71054
date_rated         71054
purchase               0
dtype: int64

## Data split

In [836]:
X = data.drop('purchase', axis=1)
y = data[['purchase']]

In [837]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=365, stratify=y)

In [838]:
#check distributions

print(y_train.value_counts(normalize=True), y_test.value_counts(normalize=True))

purchase
1           0.68957
0           0.31043
dtype: float64 purchase
1           0.689565
0           0.310435
dtype: float64


## Cleaning

1 . Inpute Missing Values
- Inpute student_country by most frequent
- Inputer date_rated by most il same in date_registered
- Inpute course_rating by most freqent 

In [839]:
#define strategy

# fit x_train and transform x_test data

x_train['date_rated'].fillna(x_train['date_registered'], inplace=True)

x_test['date_rated'].fillna(x_train['date_registered'], inplace=True)

inputer = SimpleImputer(strategy='most_frequent')

fit = inputer.fit_transform(x_train)

x_train = pd.DataFrame(data = fit, columns=inputer.feature_names_in_)
x_test  = pd.DataFrame(data = inputer.transform(x_test), columns=inputer.feature_names_in_)


In [840]:
x_train.head()

,course_id,minutes_watched,date_watched,student_country,date_registered,course_rating,date_rated
0,7,54.6,2022-01-14,US,2022-01-13,5.0,2022-01-13
1,37,0.2,2022-04-15,TN,2022-04-12,5.0,2022-04-12
2,53,35.5,2022-06-20,AE,2022-06-08,5.0,2022-06-21
3,7,6.9,2022-02-08,US,2022-01-19,5.0,2022-01-19
4,30,0.3,2022-04-19,MA,2022-04-19,5.0,2022-04-19


In [841]:
# We check if we haven't forgotten something

print("Missing values in train data : \n" , x_train.isnull().sum())
print("Missing values in test data : \n", x_train.isnull().sum())

Missing values in train data : 
 course_id          0
minutes_watched    0
date_watched       0
student_country    0
date_registered    0
course_rating      0
date_rated         0
dtype: int64
Missing values in test data : 
 course_id          0
minutes_watched    0
date_watched       0
student_country    0
date_registered    0
course_rating      0
date_rated         0
dtype: int64


In [842]:
x_train.head()

,course_id,minutes_watched,date_watched,student_country,date_registered,course_rating,date_rated
0,7,54.6,2022-01-14,US,2022-01-13,5.0,2022-01-13
1,37,0.2,2022-04-15,TN,2022-04-12,5.0,2022-04-12
2,53,35.5,2022-06-20,AE,2022-06-08,5.0,2022-06-21
3,7,6.9,2022-02-08,US,2022-01-19,5.0,2022-01-19
4,30,0.3,2022-04-19,MA,2022-04-19,5.0,2022-04-19


In [843]:
#Converte date_registered, date_watched,date_purchased to date time
cols = ['date_registered', 'date_watched', 'date_rated']
for i in cols:
    x_train[i] = pd.to_datetime(x_train[i])
    x_test[i] = pd.to_datetime(x_test[i])

In [844]:
x_train.isna().sum()

course_id          0
minutes_watched    0
date_watched       0
student_country    0
date_registered    0
course_rating      0
date_rated         0
dtype: int64

## Feature Engineering 

<p> I will create a column that will group the time between the date of recording and the date of viewing the course, 
technically  (date_registered - date_watched) </p>

In [845]:
x_train['period'] = x_train['date_watched'] - x_train['date_registered']
x_test['period'] =  x_test['date_watched'] - x_test['date_registered']


x_train['period_bf_rating'] = x_train['date_rated'] - x_train['date_watched']
x_test['period_bf_rating']  = x_test['date_rated'] - x_test['date_watched']


In [846]:
x_train.head()

,course_id,minutes_watched,date_watched,student_country,date_registered,course_rating,date_rated,period,period_bf_rating
0,7,54.6,2022-01-14,US,2022-01-13,5.0,2022-01-13,1 days,-1 days
1,37,0.2,2022-04-15,TN,2022-04-12,5.0,2022-04-12,3 days,-3 days
2,53,35.5,2022-06-20,AE,2022-06-08,5.0,2022-06-21,12 days,1 days
3,7,6.9,2022-02-08,US,2022-01-19,5.0,2022-01-19,20 days,-20 days
4,30,0.3,2022-04-19,MA,2022-04-19,5.0,2022-04-19,0 days,0 days


In [847]:
x_train.isnull().sum()

course_id           0
minutes_watched     0
date_watched        0
student_country     0
date_registered     0
course_rating       0
date_rated          0
period              0
period_bf_rating    0
dtype: int64

## Encodage

In [848]:
# We are good now, we can build and evaluate our models

In [849]:
encoder  = OrdinalEncoder()


label = ['student_country']

x_train[label] = encoder.fit_transform(x_train[label])

x_test[label] = encoder.transform(x_test[label])

x_train.isnull().sum()

ValueError: Found unknown categories ['AG', 'JE', 'PG', 'BI', 'VU'] in column 0 during transform

## Model building